В некоторых случаях необходимо объединить уже созданные индексные базы для их общего использования, посмотреть на составляющие базы, удалить отдельные чанки из векторной базы. Вот как это можно сделать:

# Устанавливаем библиотеки:

In [ ]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai==1.41.1 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
# запустите эту ячейку, если используете секретный ключ в колабе

from openai import OpenAI
import os
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import requests
import re
import tiktoken
from langchain.docstore.document import Document

# Загружаем БЗ, делим на чанки, переводим в эмбеддинги, создаем 2 векторные базы Faiss:

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url):
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# Загружаем первую БЗ
db_1=load_document_text("https://docs.google.com/document/d/1CadgV8oI_MgYZBfaIEa5vAnrQXyme1qK-qIHFwlZL58/edit?usp=sharing")
db_1

'\ufeff<Chunk>\r\nУрок: Алгоритмическое мышление - Алгоритмы поиска и сортировки\r\nСсылка https://colab.research.google.com/drive/14n-IdIQlE8mlQhA88vhxXDcC2VcuDIIt?usp=sharing\r\nАлгоритмы сортировки\r\nСуществуют десятки алгоритмов сортировки. Разные алгоритмы оптимальны\r\nдля разных наборов и типов данных. Мы рассмотрим некоторые из них:\r\n-   Пузырьковая сортировка\r\n-   Сортировка выборкой\r\n-   Сортировка вставками\r\n-   Сортировка слиянием\r\n-   Быстрая сортировка\r\n\r\n\r\nПузырьковая сортировка\r\nПузырьковая сортировка или сортировка простыми обменами – один из\r\nпростейших алгоритмов сортировки. Он применяется для упорядочивания\r\nмассивов небольших размеров.\r\n    # Алгоритм пузырьковой сортировки\r\nСуть алгоритма в том, что совершается несколько проходов по массиву. При\r\nкаждом проходе попарно сравниваются два соседних элемента. Если они\r\nнаходятся в верном порядке, то ничего не происходит, в противном случае\r\nони меняются местами. В результате первого про

In [ ]:
# Загружаем 2 БЗ:
db_2=load_document_text("https://docs.google.com/document/d/1VnnEk1-DASWaBUa5dMTeZvuAMjqgpY_K86trhCvx5TI/edit?usp=sharing")
db_2

'\ufeff<Chunk>\r\nУрок: Сверточные нейронные сети. Теоретическая часть.\r\nВведение.\r\nБиологические связи.\r\nВначале немного истории. Когда вы впервые услышали термин сверточные\r\nнейронные сети, возможно, подумали о чем-то, связанном с нейронауками\r\nили биологией, и отчасти были правы.\r\n  Сверточные нейронные сети – это своеобразная имитация зрительной коры\r\n  мозга. Зрительная кора имеет небольшие участки клеток, которые\r\n  чувствительны к конкретным областям поля зрения. Эту идею в 1962 году\r\n  детально рассмотрели Хьюбел и Визель с помощью потрясающего\r\n  эксперимента (видео).\r\n  Они показали, что отдельные мозговые нервные клетки реагировали (или\r\n  активировались) только при визуальном восприятии границ определенной\r\n  ориентации. Например, некоторые нейроны активировались, когда\r\n  воспринимали вертикальные границы, а некоторые — горизонтальные или\r\n  диагональные.\r\n  Хьюбел и Визель выяснили, что все эти нейроны организованы в блоки\r\n  стержневой а

In [ ]:
# Делим на чанки 1 БЗ:
db_1_chunks=[]
splitter = CharacterTextSplitter(separator='<Chunk>')
for chunk in splitter.split_text(db_1):
    db_1_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
print("Общее количество чанков: ", len(db_1_chunks))

Общее количество чанков:  15


In [ ]:
# Делим на чанки 2 БЗ:
db_2_chunks=[]
splitter = CharacterTextSplitter(separator='<Chunk>')
for chunk in splitter.split_text(db_2):
    db_2_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
print("Общее количество чанков: ", len(db_2_chunks))

Общее количество чанков:  6


In [ ]:
# Создаем 2 векторные базы:
embeddings = OpenAIEmbeddings()
db_1_faiss = FAISS.from_documents(db_1_chunks, embeddings)
db_2_faiss = FAISS.from_documents(db_2_chunks, embeddings)

# Объединяем две векторные базы:

In [ ]:
merged_db = FAISS.from_documents(db_1_faiss.docstore._dict.values(), embeddings)  # создаете новую базу данных на основе db1
merged_db.merge_from(db_2_faiss)  # добавляете данные из db2

Вы успешно объединили базы данных db_1_faiss и db_2_faiss в merged_db. Теперь merged_db содержит документы из обеих баз данных. Каждый документ представлен словарем, где ключом является уникальный идентификатор, а значением - объект Document, содержащий контент страницы и метаданные.

In [ ]:
# вот так можно посмотреть на объединенную базу (на все чанки)
merged_db.docstore._dict

{'a5196dfa-0c0e-491d-bca0-2dde0b0045a8': Document(metadata={'meta': 'data'}, page_content='\ufeff<Chunk>\r\nУрок: Алгоритмическое мышление - Алгоритмы поиска и сортировки\r\nСсылка https://colab.research.google.com/drive/14n-IdIQlE8mlQhA88vhxXDcC2VcuDIIt?usp=sharing\r\nАлгоритмы сортировки\r\nСуществуют десятки алгоритмов сортировки. Разные алгоритмы оптимальны\r\nдля разных наборов и типов данных. Мы рассмотрим некоторые из них:\r\n-   Пузырьковая сортировка\r\n-   Сортировка выборкой\r\n-   Сортировка вставками\r\n-   Сортировка слиянием\r\n-   Быстрая сортировка\r\n\r\n\r\nПузырьковая сортировка\r\nПузырьковая сортировка или сортировка простыми обменами – один из\r\nпростейших алгоритмов сортировки. Он применяется для упорядочивания\r\nмассивов небольших размеров.\r\n    # Алгоритм пузырьковой сортировки\r\nСуть алгоритма в том, что совершается несколько проходов по массиву. При\r\nкаждом проходе попарно сравниваются два соседних элемента. Если они\r\nнаходятся в верном порядке, то 

# Удаление отдельного чанка по его индексу

In [ ]:
# Удаление элемента по ключу и возврат его значения (опционально)
merged_db.delete(ids=['1ab77231-8cbf-4faf-803c-82ea20313dfb'])

True

In [ ]:
# проверяем - действительно, больше нет такого чанка
merged_db.docstore._dict

{'a5196dfa-0c0e-491d-bca0-2dde0b0045a8': Document(metadata={'meta': 'data'}, page_content='\ufeff<Chunk>\r\nУрок: Алгоритмическое мышление - Алгоритмы поиска и сортировки\r\nСсылка https://colab.research.google.com/drive/14n-IdIQlE8mlQhA88vhxXDcC2VcuDIIt?usp=sharing\r\nАлгоритмы сортировки\r\nСуществуют десятки алгоритмов сортировки. Разные алгоритмы оптимальны\r\nдля разных наборов и типов данных. Мы рассмотрим некоторые из них:\r\n-   Пузырьковая сортировка\r\n-   Сортировка выборкой\r\n-   Сортировка вставками\r\n-   Сортировка слиянием\r\n-   Быстрая сортировка\r\n\r\n\r\nПузырьковая сортировка\r\nПузырьковая сортировка или сортировка простыми обменами – один из\r\nпростейших алгоритмов сортировки. Он применяется для упорядочивания\r\nмассивов небольших размеров.\r\n    # Алгоритм пузырьковой сортировки\r\nСуть алгоритма в том, что совершается несколько проходов по массиву. При\r\nкаждом проходе попарно сравниваются два соседних элемента. Если они\r\nнаходятся в верном порядке, то 